In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np

# Study data files
FEMA_data = "FEMA_data.csv"
US_Presidents = "USpresidents.csv"

pd.read_csv(FEMA_data)

# Pull out year in Start Date in FEMA data 
pd.to_datetime(['Start Date'])

ParserError: Unknown string format: Start Date